In [1]:
import napari
import ioMicro as io
import numpy as np
import glob,os,sys

In [116]:
psf1 = np.load('psf_YiqunData_30-60-60.npy')

In [117]:
psf= psf1
-np.min(psf)+np.max(psf),np.sum(psf),psf.shape

(0.000113613685, 1.0000001, (30, 60, 60))

In [121]:
key = (0,1200,1200)
fl = 'psf_Bogdan_30-60-60_'+str(list(key))+'.npy'
psf2 = np.load(fl)

In [122]:
psf= psf2
-np.min(psf)+np.max(psf),np.sum(psf),psf.shape

(0.00026213427, 0.99999994, (30, 60, 60))

In [ ]:
import napari
napari.view_image(psf);

In [130]:
imtest = np.load('imtest.npy')
imtest = imtest[:450,300:,300:]

In [131]:
psf=psf2
from pycudadecon import rl_decon, rl_init, rl_cleanup, TemporaryOTF
with TemporaryOTF(psf, dzpsf=0.5, dxpsf=0.173, wavelength=545, na=1.05, nimm=1.405) as otf:
    rl_init(imtest.shape, otf.path, dzdata=0.5, dzpsf=0.5, dxdata=0.173, dxpsf=0.173)
    result=rl_decon(imtest, background=np.median(imtest[0]), n_iters=20)#,output_shape=imtest.shape)
    rl_cleanup()

In [132]:
viewer = napari.view_image(imtest)
viewer.add_image(result)

<Image layer 'result' at 0x1cdd6e71950>

In [2]:
#pickle.dump([im_PSFs,im_NPSFs],open(r'\\BBFISH7DC\Raw_data\Yiqun\im_psfs_fovs__25x_1p5.pkl','wb'))

In [97]:
import pickle
im_PSFs,im_NPSFs = pickle.load(open(r'\\BBFISH7DC\Raw_data\Yiqun\im_psfs_fovs__25x_1p5.pkl','rb'))
#im_PSFs,im_NPSFs=im_PSFs[1::2],im_NPSFs[1::2]
imNf = np.sum([imN for imP,imN in zip(im_PSFs,im_NPSFs)],axis=0)
im_PSFf = np.sum([imP*imN/imNf for imP,imN in zip(im_PSFs,im_NPSFs)],axis=0)
psf = im_PSFf[:-1,:-1,:-1]
psf -=np.min(psf)
psf = psf/np.sum(psf)
#psf/=np.max(psf)
psf = psf.astype(np.float32)
psf.shape

(60, 60, 60)

In [96]:
np.save('psf_YiqunData_30-60-60.npy',psf)

In [98]:
np.save('psf_YiqunData_60-60-60.npy',psf)

In [99]:
napari.view_image(psf)

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 29.5, 29.5), zoom=6.808333333333334, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(30.0, 1.0, 0.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=3, ndisplay=2, last_used=0, range=((0.0, 60.0, 1.0), (0.0, 60.0, 1.0), (0.0, 60.0, 1.0)), current_step=(30, 30, 30), order=(0, 1, 2), axis_labels=('0', '1', '2')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'psf' at 0x1cd9b123f50>], scale_bar=ScaleBar(visible=False, colored=False, color=array([1., 0., 1., 1.], dtype=float32), ticks=True, position=<Position.BOTTOM_RIGHT: 'bottom_right'>, font_size=10.0, box=False, box_color=array([0. , 0. , 0. , 0.6], dtype=float32), unit=None), text_overlay=TextOverlay(visible=False, color=array([0.5, 0.5, 0.5, 1. ], dtype=float32), font_size=10.0, position=<TextOverlayPosition.TOP_LEFT: 'to

In [35]:
fls = glob.glob(r'\\BBFISH7DC\Raw_data\Yiqun\Echinoderm_Lib1\Lpictus_Lib1_1dpf_Nov4_2022\Hyb2_emx1l_ets1_fgfr2-ephb3l\*.zarr')
im = io.read_im(fls[-1])
im_ = np.array(im[1],dtype=np.float32)

In [55]:
imtest = im_[:,600:900,600:900]

In [38]:
imtest.shape

(470, 900, 900)

In [39]:
np.save('imtest.npy',imtest)

In [51]:
napari.view_image(psf)

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 29.5, 29.5), zoom=6.808333333333334, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(15.0, 1.0, 0.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=3, ndisplay=2, last_used=0, range=((0.0, 30.0, 1.0), (0.0, 60.0, 1.0), (0.0, 60.0, 1.0)), current_step=(15, 30, 30), order=(0, 1, 2), axis_labels=('0', '1', '2')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'psf' at 0x1cd950c0950>], scale_bar=ScaleBar(visible=False, colored=False, color=array([1., 0., 1., 1.], dtype=float32), ticks=True, position=<Position.BOTTOM_RIGHT: 'bottom_right'>, font_size=10.0, box=False, box_color=array([0. , 0. , 0. , 0.6], dtype=float32), unit=None), text_overlay=TextOverlay(visible=False, color=array([0.5, 0.5, 0.5, 1. ], dtype=float32), font_size=10.0, position=<TextOverlayPosition.TOP_LEFT: 'to

In [3]:
import pickle
dic_psf = pickle.load(open(r'Y:\YiqunPSF\psf_Scope2_25x-1p5_cy3_shallow_small.pkl','rb'))

In [52]:
im_PSF = io.get_psf(dic_psf,key = (0, 900, 1200),sz_sm = 5,N=10,plt_val=False,thcor=0.92,slice_=slice(None))

C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\lib\nanfunctions.py:1217: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,
I:\Shared drives\YiqunWang_shared\Scripts\ioMicro.py:20: RuntimeWarning: invalid value encountered in divide
  im_PSFf = np.nansum([dic_psf[key][igood][0]*(dic_psf[key][igood][-1]/im_Nmax) for igood in igoodfovs],axis=0)


0.8349514563106796


In [104]:
for key in dic_psf.keys():
    im_PSF = io.get_psf(dic_psf,key = key,sz_sm = 5,N=10,plt_val=False,thcor=0.92,slice_=slice(None))
    name_file = 'psf_Bogdan_30-60-60_'+str(list(key))+'.npy'
    data = im_PSF
    psf = data[15:-16,1:,1:]
    psf -= np.min(psf)
    psf/=np.sum(psf)
    psf = psf.astype(np.float32)
    np.save(name_file,psf)

C:\Users\Bogdan\anaconda3\envs\cudadecon\Lib\site-packages\numpy\lib\nanfunctions.py:1217: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,
I:\Shared drives\YiqunWang_shared\Scripts\ioMicro.py:20: RuntimeWarning: invalid value encountered in divide
  im_PSFf = np.nansum([dic_psf[key][igood][0]*(dic_psf[key][igood][-1]/im_Nmax) for igood in igoodfovs],axis=0)


0.7038834951456311
0.7621359223300971
0.8058252427184466
0.8155339805825242
0.8155339805825242
0.7815533980582524
0.7718446601941747
0.7718446601941747
0.7427184466019418
0.6796116504854369
0.7524271844660194
0.7766990291262136
0.8398058252427184
0.8252427184466019
0.7961165048543689
0.8203883495145631
0.8058252427184466
0.7475728155339806
0.7766990291262136
0.7330097087378641
0.7864077669902912
0.7912621359223301
0.8252427184466019
0.8155339805825242
0.8203883495145631
0.8398058252427184
0.8106796116504854
0.7961165048543689
0.7718446601941747
0.7621359223300971
0.7766990291262136
0.7815533980582524
0.8058252427184466
0.8349514563106796
0.8349514563106796
0.8398058252427184
0.8058252427184466
0.8155339805825242
0.7766990291262136
0.7766990291262136
0.8058252427184466
0.8203883495145631
0.8155339805825242
0.8495145631067961
0.8300970873786407
0.8495145631067961
0.8252427184466019
0.8106796116504854
0.7912621359223301
0.7864077669902912
0.8203883495145631
0.8349514563106796
0.8640776699

In [ ]:
data = im_PSF
psf = data[15:-16,1:,1:]
psf -= np.min(psf)
psf/=np.sum(psf)
psf = psf.astype(np.float32)

In [6]:
from pycudadecon import rl_decon, rl_init, rl_cleanup, TemporaryOTF

In [7]:
rl_cleanup()

In [8]:
imtest = np.load('imtest.npy')

In [87]:
psf.shape

(30, 60, 60)

In [13]:
from pycudadecon import decon

In [26]:
np.min(psf)

0.0

In [70]:
data = im_PSF
psf = data[10:-11,1:,1:]
psf -= np.min(psf)
psf/=np.sum(psf)
psf = psf.astype(np.float32)

In [89]:
result3 = decon(imtest,psf)

OSError: [WinError -529697949] Windows Error 0xe06d7363

In [54]:
napari.view_image(psf)

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 29.5, 29.5), zoom=6.808333333333334, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(15.0, 1.0, 0.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=3, ndisplay=2, last_used=0, range=((0.0, 30.0, 1.0), (0.0, 60.0, 1.0), (0.0, 60.0, 1.0)), current_step=(15, 30, 30), order=(0, 1, 2), axis_labels=('0', '1', '2')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'psf' at 0x1cd95d22710>], scale_bar=ScaleBar(visible=False, colored=False, color=array([1., 0., 1., 1.], dtype=float32), ticks=True, position=<Position.BOTTOM_RIGHT: 'bottom_right'>, font_size=10.0, box=False, box_color=array([0. , 0. , 0. , 0.6], dtype=float32), unit=None), text_overlay=TextOverlay(visible=False, color=array([0.5, 0.5, 0.5, 1. ], dtype=float32), font_size=10.0, position=<TextOverlayPosition.TOP_LEFT: 'to

In [91]:
rl_cleanup()

In [62]:
psf = psf.swapaxes(0,-1)
imtest = imtest.swapaxes(0,-1)

In [72]:
psf = psf.swapaxes(0,-1)

In [94]:
from pycudadecon import rl_decon, rl_init, rl_cleanup, TemporaryOTF
with TemporaryOTF(psf, dzpsf=0.5, dxpsf=0.173, wavelength=545, na=1.05, nimm=1.405) as otf:
    rl_init(imtest.shape, otf.path, dzdata=0.5, dzpsf=0.5, dxdata=0.173, dxpsf=0.173)
    result=rl_decon(imtest, background=np.median(imtest[0]), n_iters=20)#,output_shape=imtest.shape)
    rl_cleanup()

OSError: [WinError -529697949] Windows Error 0xe06d7363

In [85]:
psf.dtype

dtype('float32')

In [74]:
viewer = napari.view_image(result)#,scale=[0.5,0.173,0.173])
viewer.add_image(imtest)#,scale=[0.5,0.173,0.173])

<Image layer 'imtest' at 0x1cdcbc00f50>

In [69]:
result.shape

(300, 300, 450)

In [80]:
imtest = imtest.swapaxes(0,-1)